## NYISO Load Prediction
- Objective: Utilize the NBEATS model to predict NYISO load data for 2023-12-31 using historical data from 2013-01-01 to 2023-12-30.
- Zones: `N.Y.C.`, `NORTH`, `CENTRL`
- Scaling methods: [definition](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html)
     - [`identity`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#std-statistics)
     - [`standard`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#std-statistics)
     - [`minmax`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#minmax-statistics)
     - [`robust`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#robust-statistics)
     - `revin`:  learnable normalization parameters are added on top of the usual normalization technique.

In [ ]:
import io
import os
import warnings
import logging
from datetime import timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.utils import io

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select, CustomJS, DatetimeTickFormatter
from bokeh.layouts import column
from bokeh.palettes import Category10

from ts_scaler.data.data_handler import DataHandler
from ts_scaler.scaler.moving_average import MovingAverageCalculator
from ts_scaler.utils.logger import setup_logger

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS, TFT
from neuralforecast.utils import AirPassengersDF

# Suppress warnings
warnings.filterwarnings("ignore")

# Suppress logging for PyTorch Lightning and set to CRITICAL level
logging.getLogger('pytorch_lightning').setLevel(logging.CRITICAL)
logging.getLogger("package").propagate = False

# Set up custom logger
logger = setup_logger(level=logging.ERROR)

# Fetch data
data_handler = DataHandler(logger=logger)
df = data_handler.fetch_nyiso_data(local_dir="../data", start_date="20130101", end_date="20231231")

# Data preprocessing
df = df.drop_duplicates(subset=['time_stamp', 'zone_name'])
df = df.dropna()
df = df[df['integrated_load'] > 0]

# Convert time_stamp to datetime
df['time_stamp'] = pd.to_datetime(df['time_stamp'])
ndf = df.rename(columns={"time_stamp": "ds", "integrated_load": "y", "zone_name": "unique_id"})[["ds", "unique_id", "y"]]

# Filter data by unique_id
nyc_df = ndf[ndf.unique_id == "N.Y.C."]
north_df = ndf[ndf.unique_id == "NORTH"]
central_df = ndf[ndf.unique_id == "CENTRL"]

In [17]:
%%capture

def train_and_prediction(df, zone):
    df_test = df.tail(24)
    df_train = df.drop(df_test.index)
    
    horizon = len(df_test)
    models = [
        NBEATS(input_size=len(df_test) * 7, h=horizon, max_steps=500, scaler_type='identity'),
        NBEATS(input_size=len(df_test) * 7, h=horizon, max_steps=500, scaler_type='standard'),
        NBEATS(input_size=len(df_test) * 7, h=horizon, max_steps=500, scaler_type='minmax'),
        NBEATS(input_size=len(df_test) * 7, h=horizon, max_steps=500, scaler_type='robust'),
        NBEATS(input_size=len(df_test) * 7, h=horizon, max_steps=500, scaler_type='revin'),
    ]

    nf = NeuralForecast(models=models, freq='H')
    nf.fit(df=df_train)
    Y_hat_df = nf.predict().reset_index()

    Y_hat_df = df_test.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
    
    plot_df = pd.concat([df_train[-(24 * 3):], Y_hat_df]).drop("unique_id", axis=1).set_index('ds').rename(columns={
        'NBEATS': 'NBEATS - Identity',
        'NBEATS1': 'NBEATS - Standard',
        'NBEATS2': 'NBEATS - MinMax',
        'NBEATS3': 'NBEATS - Robust',
        'NBEATS4': 'NBEATS - Revin',    
    })
    plot_df.index = pd.to_datetime(plot_df.index)
    return plot_df

def plot_prediction(plot_df, zone):
    source = ColumnDataSource(plot_df)
    
    p = figure(title=f"NYISO - {zone}", x_axis_type='datetime', x_axis_label='Timestamp [t]', y_axis_label='Load', width=1200, height=600)
    
    colors = Category10[len(plot_df.columns)]
    
    for i, col in enumerate(plot_df.columns):
        p.line(x='ds', y=col, source=source, line_width=2, color=colors[i], legend_label=col)
    
    p.legend.title = ''
    p.legend.title_text_font_size = '12pt'
    p.legend.label_text_font_size = '10pt'
    p.legend.location = 'top_left'
    
    p.xaxis.formatter = DatetimeTickFormatter(
        days="%Y-%m-%d",
        months="%Y-%m-%d",
        years="%Y-%m-%d"
    )
    
    output_notebook()
    show(p)



with io.capture_output() as captured:
    nyc_prediction_df = train_and_prediction(nyc_df, "NYC")
    north_prediction_df = train_and_prediction(north_df, "NORTH")
    centrl_prediction_df = train_and_prediction(central_df, "CENTRL")

In [19]:
plot_prediction(nyc_prediction_df, "NYC")

Loading BokehJS ...

In [20]:
plot_prediction(north_prediction_df, "NORTH")

Loading BokehJS ...

In [21]:
plot_prediction(centrl_prediction_df, "CENTRL")

Loading BokehJS ...